In [2]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import QCNNHybrid
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import numpy as np

import matplotlib.pyplot as plt
import time

2022-09-25 18:07:25.311213: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-25 18:07:25.311249: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-25 18:07:30.933361: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-25 18:07:30.933454: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-09-25 18:07:30.934025: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [4]:
args = argparse.Namespace()

# Data
args.resize = [8,8]
args.standardize = 1
args.binary_data = [0,1]
args.batch_size = 32
args.validation_split = 0.05
args.labels_to_categorical = 1
args.opt = 'Ranger'

# Base Model
args.learning_rate = 1e-3
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 1
args.ansatz = "NQubit"
args.sparse = 1

In [5]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Resizing data...
Resizing data...
Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤══════════════════╤════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size       │ Test size       │ Dims      │
╞════════╪══════════════════╪════════════════╪═════════════════╪═══════════╡
│ X      │ (12031, 8, 8, 1) │ (634, 8, 8, 1) │ (2115, 8, 8, 1) │ (8, 8, 1) │
├────────┼──────────────────┼────────────────┼─────────────────┼───────────┤
│ y      │ (12031, 2)       │ (634, 2)       │ (2115, 2)       │ (2,)      │
╘════════╧══════════════════╧════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤════════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │    Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪════════╪════════╪═══════╪══════════════════════════╡
│ Train Images │ -1.58 │ 112.53 │      0 │  

In [6]:
data.config()

{'input_dims': (8, 8, 1), 'output_dims': (2,), 'mapping': [0, 1]}

In [7]:
model = QCNNHybrid(data.config(), args)

../../qml_hep_lhc/layers/qconv2d.py:66: UserWarning: fm_class is only used for TwoLayerPQC.
  warnings.warn('fm_class is only used for TwoLayerPQC.')
../../qml_hep_lhc/layers/qconv2d.py:70: UserWarning: ansatz_class is only used for TwoLayerPQC.
  warnings.warn('ansatz_class is only used for TwoLayerPQC.')


In [8]:
model.compile()
model.build_graph().summary()

Model: "QCNNHybrid-AngleMap-NQubit"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8, 8, 1)]         0         
                                                                 
 qconv2d_1 (QConv2D)         (None, 8, 8, 1, 1)        30        
                                                                 
 qconv2d_2 (QConv2D)         (None, 8, 8, 1, 1)        15        
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 8)                 520       
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 2)  

In [9]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]
model.fit(data, callbacks)

Epoch 1/10
376/376 [==============================] - 276s 732ms/step - loss: 0.3201 - auc: 0.9393 - accuracy: 0.8634 - val_loss: 0.1237 - val_auc: 0.9903 - val_accuracy: 0.9574 - lr: 0.0010
Epoch 2/10
376/376 [==============================] - 275s 732ms/step - loss: 0.1036 - auc: 0.9930 - accuracy: 0.9655 - val_loss: 0.1179 - val_auc: 0.9913 - val_accuracy: 0.9558 - lr: 0.0010
Epoch 3/10
376/376 [==============================] - 266s 708ms/step - loss: 0.0834 - auc: 0.9949 - accuracy: 0.9723 - val_loss: 0.1039 - val_auc: 0.9925 - val_accuracy: 0.9637 - lr: 0.0010
Epoch 4/10
376/376 [==============================] - 267s 711ms/step - loss: 0.0741 - auc: 0.9956 - accuracy: 0.9755 - val_loss: 0.0956 - val_auc: 0.9948 - val_accuracy: 0.9606 - lr: 0.0010
Epoch 5/10
376/376 [==============================] - 282s 750ms/step - loss: 0.0702 - auc: 0.9960 - accuracy: 0.9775 - val_loss: 0.0929 - val_auc: 0.9955 - val_accuracy: 0.9637 - lr: 0.0010
Epoch 6/10
376/376 [=========================

In [10]:
model.test(data, callbacks=callbacks)

67/67 [==============================] - 27s 403ms/step - loss: 0.0462 - auc: 0.9983 - accuracy: 0.9820


[0.046235933899879456,
 0.998317301273346,
 0.9820330739021301,
 0.998317301273346]